In [3]:
from torch import nn

class NetworkInNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            #stage1
            nn.Conv2(3,192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2(192,160, kernel_size=1),
            nn.ReLU(),
            nn.Conv2(192,92, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, strode=2, cell_mode=True),
            nn.Dropout(),
            #stage2
            nn.Conv2(92,192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2(192,192, kernel_size=1),
            nn.ReLU(),
            nn.Conv2(192,192, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, strode=2, cell_mode=True),
            nn.Dropout(),
            #stage3
            nn.Conv2(192,192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2(192,192, kernel_size=1),
            nn.ReLU(),
            nn.Conv2(192,1000, kernel_size=1),
            nn.ReLU(),
            # ? Avgpool과 maxpool의 차이
            nn.AvgPool2d(kernel_size=8, strode=1),
        )
    def forward(self, x):
        x = self.feature_extractortractor(x)
        x = x.view(x.size(0), 1000)
        return x
        

# ResNet




In [8]:
from typing import Tuple, List, Optional, Callable

import torch
import torch.nn 
#활성화 함수는 학습하는 것이 아님
# 굳이 nn으로 넣어줄 필요없이 그냥 꺼내서 써도 된다.
from torch import nn, Tensor
import torch.nn.functional as F

class Conv2d1x1(nn.Sequential):
    def __init__(
        self,
        in_planes : int,
        out_planes : int,
        stride : int = 1,
    ) -> None:
        super().__init__(
            nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)
        )

class basicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=False):
        supur().__init()
        self.conv1 = nn.Comv2d(in_channels,out_channels,kernel_size=3,stride=1,bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Comv2d(out_channels,out_channels,kernel_size=3,stride=1,bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()
        
        self.downsample = None
        if downsample:
        
        
    def forward(self, x):
        original = x
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        
        x += original
        x = self.relu2(x)
        
        return x
    
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1, downsample=None
        super().__init__()
        self.conv1 = Conv2d1x1(in_channels, out_channels)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = Conv2d3x3(in_channels, out_channels, stride=stride)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.conv3 = Conv2d1x1(in_channels, out_channels * self.expansion)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)

        self.downsample = downsample
                 

    def forward(self, x):
        original = x
        x = F.relu(self.bn1(self.conv1(x))) #First Layer
        x = F.relu(self.bn2(self.conv2(x))) #second Layer
        x = self.bn3(self.conv1(x)) #First Layer
                 

        if self.downsample is not None:
            residual = self.downsample(inputs)

        outputs += residual
        outputs = F.relu(outputs, inplace=True)

        return outputs
    
class ResNet(nn.Module):
    def __init__(self, layers, block):
        super().__init__()
        self.in_channels = 64
        
        self.stem = nn.Sequ():
        
            self.conv = nn.Conv2(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
            self.bn = nn.BatchNorm2d(64)
            self.relu = nn.ReLU()
            self.maxpool2d = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.stage1 = self._make_layers(block, 64, layers[0])
        #아래부터는 padding이 들어가므로 stride를 넣어줌.
        self.stage2 = self._make_layers(block, 128, layers[1], stride=2)
        self.stage3 = self._make_layers(block, 256, layers[2], stride=2)
        self.stage4 = self._make_layers(block, 512, layers[3], stride=2)
        
    def _make_layers(self, block, channels, num_layers, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d1x1(
                    self.in_channels, 
                    channels * block.expansion,
                    stride=stride
                ),
                nn.BatchNorm2d(channels * block.expansion)
            )
            
        layers = []
        for _ in range(1, num_blocks):
            layers.append(block(in_channels, out_chanenls))
            
            nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        return x
        
model = ResNet(BasicBlock, [3,4,36,3], BottleneckBlock) #ResNet-34
print(summary(model, input_size=(1,3,224,224)))

#이해해야 할 것 : 설계는 계속 변경될 수 있다.
#에러1. 차원이 달라지면 덧셈연산이 안된다.
#에러2. 블럭마다 다운 샘플링을 하면 안된다. 마지막에 하거나 ?
#에러3. asnet 18, 34, 50, 101 -> 블록이 2가지있다. 베이직과 ?블럭이 있는데 이거만 교체하고, 숫자만 교체하면 코딩을 다시 해야한다.

# 콘가 뭐임?
#init가 뭐임?
# jupyter lab 여러줄 주석단축키가 뭐임?
#레이어가 뭐임?
                                        

SyntaxError: invalid syntax (Temp/ipykernel_20208/3424810190.py, line 3)